In [364]:
import os
import torch

base_folder = "runs/"
stamp = "20231005-0443-1"
folder = os.path.join(base_folder, stamp)
data = []
for i in range(100):
    path = os.path.join(folder, f"{i}.pt")
    if os.path.exists(path):
       data.append(torch.load(path))

In [151]:
import importlib
import src.EINCASM as EINCASM
importlib.reload(EINCASM)

ein = EINCASM.EINCASM("config.yaml")

TypeError: Simulation.add_channel() got an unexpected keyword argument 'metadata'

In [149]:
ein.sim.init_all_channels()

In [150]:
ein.sim.channels

[]